In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import torch
from torch.utils.data import Subset, ConcatDataset
from pathlib import Path
import numpy as np
from lib import *
from read_eeg import create_datasets, collate_fn

base_dir = Path("/ocean/projects/cis240129p/shared/data/eeg_alice")
subjects_used = ["S04"]  # exclude 'S05' - less channels # , "S13", "S19"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
ds = BrennanSeqDataset(
    root_dir=base_dir,
    phoneme_dir=base_dir / "phonemes",
    idx="S04",
    seq_len=5,
    phoneme_dict_path=base_dir / "phoneme_dict.txt",
    debug=False,
)

Extracting parameters from /ocean/projects/cis240129p/shared/data/eeg_alice/S04.vhdr...
Setting channel info structure...
Reading 0 ... 368449  =      0.000 ...   736.898 secs...


In [25]:
item = ds[0]
item.keys()

0 Alice was beginning to get


dict_keys(['labels', 'audio_feats', 'audio_raw', 'eeg_raw', 'eeg_feats', 'phonemes'])

In [29]:
for i, data in enumerate(ds):
    # print data shape
    print(
        f"{data['audio_feats'].shape}, {data['eeg_raw'].shape},{data['eeg_feats'].shape}, {data['phonemes'].shape}"
    )
    print(i, data["labels"])
    if i > 10:
        break

(624, 128), (2600, 60),(795, 300), (624,)
0 Alice was beginning to get
(650, 128), (2600, 60),(795, 300), (650,)
1 very tired of sitting by
(650, 128), (2600, 60),(795, 300), (650,)
2 her sister on the bank
(650, 128), (2600, 60),(795, 300), (650,)
3 and of having nothing to
(650, 128), (2600, 60),(795, 300), (650,)
4 do once or twice she
(650, 128), (2600, 60),(795, 300), (650,)
5 peeped into the book her
(650, 128), (2600, 60),(795, 300), (650,)
6 sister was reading but it
(650, 128), (2600, 60),(795, 300), (650,)
7 had no pictures or conversations
(650, 128), (2600, 60),(795, 300), (650,)
8 in it and what s
(650, 128), (2600, 60),(795, 300), (650,)
9 the use of a book
(650, 128), (2600, 60),(795, 300), (650,)
10 thought Alice without pictures or
(650, 128), (2600, 60),(795, 300), (650,)
11 conversation So she was considering


In [14]:
len(item["audio_feats"])

5

In [ ]:
item2 = ds_brennan[0]
print(item2.keys())
print(item2["audio_feats"].shape, item2["eeg_feats"].shape, len(item2["phonemes"]))

0 Alice
dict_keys(['label', 'audio_feats', 'audio_raw', 'eeg_raw', 'eeg_feats', 'phonemes'])
(104, 128) (159, 300) 104


In [6]:
item["audio_feats"][0].shape, item["eeg_feats"][0].shape, len(item["phonemes"][0])

((104, 128), (159, 300), 104)

In [ ]:
ds_brennan = BrennanDataset(
    root_dir=base_dir,
    phoneme_dir=base_dir / "phonemes",
    idx="S04",
    phoneme_dict_path=base_dir / "phoneme_dict.txt",
    debug=True,
)

Extracting parameters from /ocean/projects/cis240129p/shared/data/eeg_alice/S04.vhdr...
Setting channel info structure...
Reading 0 ... 368449  =      0.000 ...   736.898 secs...


In [11]:
item2 = ds_brennan[0]
print(item2.keys())
print(item2["audio_feats"].shape, item2["eeg_feats"].shape, len(item2["phonemes"]))

0 Alice
dict_keys(['label', 'audio_feats', 'audio_raw', 'eeg_raw', 'eeg_feats', 'phonemes'])
(104, 128) (159, 300) 104


In [16]:
train_dataset, test_dataset = create_datasets(subjects_used, base_dir)

print(
    f"Train dataset length: {len(train_dataset)}, Test dataset length: {len(test_dataset)}"
)

Extracting parameters from /ocean/projects/cis240129p/shared/data/eeg_alice/S04.vhdr...
Setting channel info structure...
Reading 0 ... 368449  =      0.000 ...   736.898 secs...


Train dataset length: 1703, Test dataset length: 426


In [17]:
# def collate_fn(batch):
#     """
#     A custom collate function that handles different types of data in a batch.
#     It dynamically creates batches by converting arrays or lists to tensors and
#     applies padding to variable-length sequences.
#     """
#     batch_dict = {}
#     for key in batch[0].keys():
#         batch_items = [item[key] for item in batch]
#         if isinstance(batch_items[0], np.ndarray) or isinstance(
#             batch_items[0], torch.Tensor
#         ):
#             if isinstance(batch_items[0], np.ndarray):
#                 batch_items = [torch.tensor(b) for b in batch_items]
#             if len(batch_items[0].shape) > 0:
#                 batch_dict[key] = torch.nn.utils.rnn.pad_sequence(
#                     batch_items, batch_first=True  # pad with zeros
#                 )
#             else:
#                 batch_dict[key] = torch.stack(batch_items)
#         else:
#             batch_dict[key] = batch_items

#     return batch_dict


train_dataloder = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=2,
    num_workers=1,
    shuffle=True,
    collate_fn=collate_fn,
)

test_dataloder = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=2,
    num_workers=1,
    shuffle=False,
    collate_fn=collate_fn,
)

In [18]:
item = train_dataset[0]
for k, v in item.items():
    try:
        print(k, v.shape, type(v))
    except:
        print(k, type(v))

label <class 'str'>
audio_feats (104, 128) <class 'numpy.ndarray'>
audio_raw (16735,) <class 'numpy.ndarray'>
eeg_raw (520, 62) <class 'numpy.ndarray'>
eeg_feats (159, 310) <class 'numpy.ndarray'>
phonemes (104,) <class 'numpy.ndarray'>


In [23]:
item["eeg_raw"].dtype

dtype('float64')

In [19]:
# test dataloader
i = 0
for batch in train_dataloder:
    print(i)
    for k, v in batch.items():
        try:
            print(k, v.shape, type(v))
        except:
            print(k, type(v))
    i += 1
    if i > 4:
        break

0
label <class 'list'>
audio_feats torch.Size([2, 130, 128]) <class 'torch.Tensor'>
audio_raw torch.Size([2, 20800]) <class 'torch.Tensor'>
eeg_raw torch.Size([2, 520, 62]) <class 'torch.Tensor'>
eeg_feats torch.Size([2, 159, 310]) <class 'torch.Tensor'>
phonemes torch.Size([2, 130]) <class 'torch.Tensor'>
1
label <class 'list'>
audio_feats torch.Size([2, 130, 128]) <class 'torch.Tensor'>
audio_raw torch.Size([2, 20800]) <class 'torch.Tensor'>
eeg_raw torch.Size([2, 520, 62]) <class 'torch.Tensor'>
eeg_feats torch.Size([2, 159, 310]) <class 'torch.Tensor'>
phonemes torch.Size([2, 130]) <class 'torch.Tensor'>
2
label <class 'list'>
audio_feats torch.Size([2, 130, 128]) <class 'torch.Tensor'>
audio_raw torch.Size([2, 20800]) <class 'torch.Tensor'>
eeg_raw torch.Size([2, 520, 62]) <class 'torch.Tensor'>
eeg_feats torch.Size([2, 159, 310]) <class 'torch.Tensor'>
phonemes torch.Size([2, 130]) <class 'torch.Tensor'>
3
label <class 'list'>
audio_feats torch.Size([2, 130, 128]) <class 'torch.T